# Family-friendly Places in Munich

## Introduction

This notebook is my capstone project assignment for the [IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science) course.
For this assignment each student could come up with a problem, that could be solved by leveraging Foursquare location data and machine learning algorithms.

## Business Problem

Because I live in Munich, Germany and I have a family, I decided to explore districts in Munich and compare them by family friendly food places (restaurants, bakeries etc.).
Not every place in a city is family friendly. This project can help analyze Munich from that perspective and help families to explore the city, or maybe even settle in one of the neighborhoods.
Audience of this project are families who either live in Munich or visit the city.

## Data

In this section I’m getting the required data for the analysis and show examples of the data.
The data, required for this project is:

- List of Munich districts and their coordinates.
- List of family friendly places with coordinates.

#### Sources

- List of Munich districts: https://www.toytowngermany.com/wiki/Munich_city_districts
- District locations are obtained via `geopy` library based on district name.
- Family friendly places are obtained via Foursquare API.

Munich has the following districts:

In [2]:
district_names = [
    'Altstadt - Lehel',
    'Ludwigsvorstadt - Isarvorstadt',
    'Maxvorstadt',
    'Schwabing-West',
    'Au - Haidhausen',
    'Sendling',
    'Sendling - Westpark',
    'Schwanthalerhöhe',
    'Neuhausen - Nymphenburg',
    'Moosach',
    'Milbertshofen - Am Hart',
    'Schwabing - Freimann',
    'Bogenhausen',
    'Berg am Laim',
    'Trudering - Riem',
    'Ramersdorf - Perlach',
    'Obergiesing',
    'Untergiesing - Harlaching',
    'Thalkirchen - Obersendling - Forstenried - Fürstenried - Solln',
    'Hadern',
    'Pasing - Obermenzing',
    'Aubing - Lochhausen - Langwied',
    'Allach - Untermenzing',
    'Feldmoching - Hasenbergl',
    'Laim'
]

Let’s install required dependencies to explore the districts

In [90]:
len(district_names)

25

In [1]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.



In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.



Importing packages required for maps

In [3]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# map rendering library
import folium

Getting the location of Munich

In [4]:
city_address = 'Munich, Germany'

geolocator = Nominatim(user_agent='munich_explorer')
location = geolocator.geocode(city_address)
city_latitude = location.latitude
city_longitude = location.longitude

print(f'The geograpical coordinate of Munich are {city_latitude}, {city_longitude}.')

The geograpical coordinate of Munich are 48.1371079, 11.5753822.


Getting the location of each district

In [5]:
get_location = lambda name: geolocator.geocode(f'{name}, Munich, Germany')
district_locations = []
for name in district_names:
    loc = get_location(name)
    if loc == None:
        print(f'Could not find location for {name}')
    else:
        district_locations.append((name, loc.latitude, loc.longitude))
district_locations

[('Altstadt - Lehel', 48.1378285, 11.5745823),
 ('Ludwigsvorstadt - Isarvorstadt', 48.1317712, 11.5558087),
 ('Maxvorstadt', 48.1510916, 11.5624179),
 ('Schwabing-West', 48.1648489, 11.5634997),
 ('Au - Haidhausen', 48.1287531, 11.5905362),
 ('Sendling', 48.1180125, 11.5390832),
 ('Sendling - Westpark', 48.11803085, 11.519332770284128),
 ('Schwanthalerhöhe', 48.1337822, 11.5410566),
 ('Neuhausen - Nymphenburg', 48.1542217, 11.5315172),
 ('Moosach', 48.1798949, 11.5105712),
 ('Milbertshofen - Am Hart', 48.1823848, 11.5750432),
 ('Schwabing - Freimann', 48.1702395, 11.5853886),
 ('Bogenhausen', 48.1547823, 11.6334838),
 ('Berg am Laim', 48.1234833, 11.6334511),
 ('Trudering - Riem', 48.1260355, 11.6633383),
 ('Ramersdorf - Perlach', 48.1141401, 11.6142551),
 ('Obergiesing', 48.1111557, 11.5889093),
 ('Untergiesing - Harlaching', 48.1149632, 11.5701894),
 ('Thalkirchen - Obersendling - Forstenried - Fürstenried - Solln',
  48.1028401,
  11.5459789),
 ('Hadern', 48.118064, 11.4818417),
 ('

Let’s take a look at Munich and its districts on the map

In [6]:
munich_map = folium.Map(location=[city_latitude, city_longitude], zoom_start=11.5)

for name, lat, lng in district_locations:
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(munich_map)  
    
munich_map

Let’s connect Foursqueare API to explore location data. I’ll be using Foursquare API.
Since this notebook is public, I’m gonna store the credentials locally in a separate file.

In [7]:
from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('secrets.cfg')

foursquare_client_id = parser.get('secrets', 'foursquare_client_id')
foursquare_client_secret = parser.get('secrets', 'foursquare_client_secret')
foursquare_version = '20180605'

The idea is to search for venues in category `food`, that are marked as `family friendly`. To do that, I’m gonna use `venues/explore` API and specify `family friendly` in the query:

In [8]:
import requests
import pandas as pd

venues_list = []
categoryId = '4d4b7105d754a06374d81259' # food category
for name, lat, lng in district_locations:
    # create the API request URL
    url = (f'https://api.foursquare.com/v2/venues/explore?&query=family+friendly'
            f'&client_id={foursquare_client_id}&client_secret={foursquare_client_secret}'
            f'&v={foursquare_version}&categoryId={categoryId}&ll={lat},{lng}&limit=1000')

    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']

    # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])
    
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

In [52]:
nearby_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt - Lehel,48.137828,11.574582,Augustiner Klosterwirt,48.138649,11.572527,German Restaurant
1,Altstadt - Lehel,48.137828,11.574582,Andechser am Dom,48.138302,11.573778,Bavarian Restaurant
2,Altstadt - Lehel,48.137828,11.574582,Kilians Irish Pub,48.138720,11.574556,Irish Pub
3,Altstadt - Lehel,48.137828,11.574582,La Burrita,48.136143,11.574489,Burrito Place
4,Altstadt - Lehel,48.137828,11.574582,Chocolaterie Beluga,48.135750,11.575776,Café
5,Altstadt - Lehel,48.137828,11.574582,Nürnberger Bratwurst Glöckl am Dom,48.138191,11.574165,Bavarian Restaurant
6,Altstadt - Lehel,48.137828,11.574582,Bite Delite,48.139996,11.575072,Café
7,Altstadt - Lehel,48.137828,11.574582,Restaurant Dallmayr,48.138489,11.576791,German Restaurant
8,Altstadt - Lehel,48.137828,11.574582,CrêpeFruit,48.138478,11.576857,Creperie
9,Altstadt - Lehel,48.137828,11.574582,Ratskeller,48.137583,11.576337,Bavarian Restaurant


What’s our total amount of venue categories?

In [54]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 98 uniques categories.


Let’s see them:

In [55]:
nearby_venues['Venue Category'].unique()

array(['German Restaurant', 'Bavarian Restaurant', 'Irish Pub',
       'Burrito Place', 'Café', 'Creperie', 'Seafood Restaurant',
       'Ice Cream Shop', 'Restaurant', 'Pizza Place',
       'Falafel Restaurant', 'Snack Place', 'Italian Restaurant',
       'Food Court', 'Afghan Restaurant', 'Soup Place',
       'Vegetarian / Vegan Restaurant', 'Theme Restaurant',
       'Sandwich Place', 'Steakhouse', 'Cupcake Shop',
       'English Restaurant', 'Argentinian Restaurant', 'Salad Place',
       'Burger Joint', 'French Restaurant', 'Portuguese Restaurant',
       'Mediterranean Restaurant', 'Trattoria/Osteria', 'Manti Place',
       'Japanese Restaurant', 'Breakfast Spot', 'Bakery',
       'Modern European Restaurant', 'Asian Restaurant',
       'Xinjiang Restaurant', 'Vietnamese Restaurant', 'Greek Restaurant',
       'BBQ Joint', 'Middle Eastern Restaurant', 'Spanish Restaurant',
       'Caucasian Restaurant', 'African Restaurant', 'Chinese Restaurant',
       'Mexican Restaurant', 'Tur

Using the obtained data on family friendly food places, I’m gonna compare districts of Munich, applying K-means clustering approach.

## Methodology

In this project I’m providing families with information on Munich districts and helping to compare them.
As a first step I’m going to analyze the most common venues in each district.
Then I’m going to apply K-means clustering to put the districts in clusters for better comparison.
Finally I’m going to compare each cluster.

## Analysis

### Analyzing districts

Let’s see what venue categories exist in each district.

In [11]:
venues_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix='', prefix_sep='')
venues_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Altstadt - Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
venues_onehot.shape

(2500, 99)

Now let’s group venue occurances:

In [12]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,Allach - Untermenzing,0.00,0.00,0.02,0.00,0.03,0.00,0.00,0.00,0.08,...,0.00,0.05,0.01,0.01,0.00,0.07,0.01,0.01,0.01,0.00
1,Altstadt - Lehel,0.02,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,...,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.00
2,Au - Haidhausen,0.02,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.04,...,0.01,0.01,0.01,0.01,0.00,0.01,0.03,0.02,0.03,0.00
3,Aubing - Lochhausen - Langwied,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.10,...,0.00,0.02,0.02,0.01,0.00,0.03,0.01,0.01,0.01,0.00
4,Berg am Laim,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.08,...,0.00,0.02,0.02,0.00,0.00,0.04,0.01,0.02,0.01,0.00
5,Bogenhausen,0.01,0.00,0.00,0.00,0.02,0.02,0.01,0.01,0.04,...,0.00,0.01,0.02,0.00,0.00,0.04,0.00,0.00,0.03,0.00
6,Feldmoching - Hasenbergl,0.00,0.00,0.02,0.00,0.03,0.01,0.00,0.00,0.07,...,0.00,0.02,0.02,0.00,0.00,0.08,0.01,0.01,0.01,0.00
7,Hadern,0.00,0.00,0.01,0.00,0.03,0.01,0.01,0.01,0.06,...,0.01,0.00,0.04,0.00,0.00,0.01,0.01,0.01,0.02,0.00
8,Laim,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.01,0.03,...,0.00,0.01,0.01,0.00,0.00,0.04,0.01,0.00,0.02,0.00
9,Ludwigsvorstadt - Isarvorstadt,0.00,0.01,0.01,0.01,0.04,0.01,0.01,0.00,0.02,...,0.01,0.00,0.02,0.00,0.00,0.00,0.03,0.01,0.05,0.01


The data listed above already gives some idea of the kind of places we can see in each district.
Now we can see top 5 places per district:

In [75]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allach - Untermenzing,German Restaurant,Italian Restaurant,Bakery,Greek Restaurant,Trattoria/Osteria
1,Altstadt - Lehel,Café,Bavarian Restaurant,German Restaurant,Restaurant,Italian Restaurant
2,Au - Haidhausen,Café,Italian Restaurant,German Restaurant,French Restaurant,Bavarian Restaurant
3,Aubing - Lochhausen - Langwied,German Restaurant,Italian Restaurant,Bakery,Greek Restaurant,Ice Cream Shop
4,Berg am Laim,Italian Restaurant,German Restaurant,Bakery,Café,Indian Restaurant
5,Bogenhausen,Italian Restaurant,German Restaurant,Café,Restaurant,Trattoria/Osteria
6,Feldmoching - Hasenbergl,German Restaurant,Greek Restaurant,Café,Trattoria/Osteria,Italian Restaurant
7,Hadern,Café,Ice Cream Shop,German Restaurant,Greek Restaurant,Italian Restaurant
8,Laim,Café,Italian Restaurant,German Restaurant,Greek Restaurant,Ice Cream Shop
9,Ludwigsvorstadt - Isarvorstadt,Café,Italian Restaurant,German Restaurant,Burger Joint,Vietnamese Restaurant


We can see in the data that some districts in Munich are similar and have common top venues, although not all districts are similar. Café, German Restaurant and Italian Restaurant seem to be very common.

Overall we can distinguish 5 most common venues:

In [71]:
nearby_venues['Venue Category'].value_counts()

Café                         277
Italian Restaurant           277
German Restaurant            236
Greek Restaurant             138
Bakery                       115
                            ... 
South American Restaurant      1
Peruvian Restaurant            1
Dim Sum Restaurant             1
Hot Dog Joint                  1
Creperie                       1
Name: Venue Category, Length: 98, dtype: int64

### Clustering

In this section I’m gonna apply K-means clustering to group the districts into clusters. First of all let’s put the district location data into a dataframe:

In [44]:
district_data = pd.DataFrame(district_locations, columns=['Neighborhood', 'Latitude', 'Longitude'])
district_data

,Neighborhood,Latitude,Longitude
0,Altstadt - Lehel,48.137828,11.574582
1,Ludwigsvorstadt - Isarvorstadt,48.131771,11.555809
2,Maxvorstadt,48.151092,11.562418
3,Schwabing-West,48.164849,11.563500
4,Au - Haidhausen,48.128753,11.590536
5,Sendling,48.118012,11.539083
6,Sendling - Westpark,48.118031,11.519333
7,Schwanthalerhöhe,48.133782,11.541057
8,Neuhausen - Nymphenburg,48.154222,11.531517
9,Moosach,48.179895,11.510571


Now we can do some clustering:

In [76]:
from sklearn.cluster import KMeans

kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(venues_grouped_clustering)
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
venues_merged = district_data
venues_merged = venues_merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

venues_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Altstadt - Lehel,48.137828,11.574582,2,Café,Bavarian Restaurant,German Restaurant,Restaurant,Italian Restaurant
1,Ludwigsvorstadt - Isarvorstadt,48.131771,11.555809,2,Café,Italian Restaurant,German Restaurant,Burger Joint,Vietnamese Restaurant
2,Maxvorstadt,48.151092,11.562418,2,Café,Italian Restaurant,Restaurant,German Restaurant,Vietnamese Restaurant
3,Schwabing-West,48.164849,11.563500,0,Café,Italian Restaurant,Vietnamese Restaurant,Asian Restaurant,Trattoria/Osteria
4,Au - Haidhausen,48.128753,11.590536,2,Café,Italian Restaurant,German Restaurant,French Restaurant,Bavarian Restaurant
5,Sendling,48.118012,11.539083,4,Café,Italian Restaurant,Ice Cream Shop,German Restaurant,Greek Restaurant
6,Sendling - Westpark,48.118031,11.519333,4,Café,Italian Restaurant,Greek Restaurant,Ice Cream Shop,Asian Restaurant
7,Schwanthalerhöhe,48.133782,11.541057,2,Café,Italian Restaurant,German Restaurant,Asian Restaurant,Ice Cream Shop
8,Neuhausen - Nymphenburg,48.154222,11.531517,3,Italian Restaurant,German Restaurant,Café,Bakery,Sushi Restaurant
9,Moosach,48.179895,11.510571,3,Italian Restaurant,German Restaurant,Greek Restaurant,Café,Bakery


Ok, let’s visualize our clusters:

In [84]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[city_latitude, city_longitude], zoom_start=11.3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Neighborhood'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Having our 5 clusters on the map, we can see one interesting observation: the districts within a cluster aren’t necesserily close to each other geographically. For example districts in `Cluster 1` and `Cluster 3` are not in the same parts of the city.

Let’s do further analysis of each cluster to see their key features.

In [38]:
def show_cluster(cluster_id):
    return venues_merged.loc[venues_merged['Cluster Labels'] == cluster_id, venues_merged.columns[[0] + list(range(4, venues_merged.shape[1]))]]

#### Cluster 0

In [78]:
show_cluster(0)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Schwabing-West,Café,Italian Restaurant,Vietnamese Restaurant,Asian Restaurant,Trattoria/Osteria
10,Milbertshofen - Am Hart,Italian Restaurant,Café,Thai Restaurant,Greek Restaurant,Bakery
11,Schwabing - Freimann,Café,Italian Restaurant,Vietnamese Restaurant,Trattoria/Osteria,Thai Restaurant


In districts of this cluster families will mostly find: `cafe`, `italian restaurant` and some kind of `asian restaurant`.

#### Cluster 1

In [79]:
show_cluster(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Berg am Laim,Italian Restaurant,German Restaurant,Bakery,Café,Indian Restaurant
14,Trudering - Riem,Italian Restaurant,German Restaurant,Ice Cream Shop,Bakery,Greek Restaurant
20,Pasing - Obermenzing,German Restaurant,Italian Restaurant,Greek Restaurant,Bakery,Ice Cream Shop
21,Aubing - Lochhausen - Langwied,German Restaurant,Italian Restaurant,Bakery,Greek Restaurant,Ice Cream Shop
22,Allach - Untermenzing,German Restaurant,Italian Restaurant,Bakery,Greek Restaurant,Trattoria/Osteria


Looks like a restaurant focused cluster. In districts of this cluster families will mostly find: `italian restaurant`, `german restaurant` and `bakery`.

#### Cluster 2

In [80]:
show_cluster(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Altstadt - Lehel,Café,Bavarian Restaurant,German Restaurant,Restaurant,Italian Restaurant
1,Ludwigsvorstadt - Isarvorstadt,Café,Italian Restaurant,German Restaurant,Burger Joint,Vietnamese Restaurant
2,Maxvorstadt,Café,Italian Restaurant,Restaurant,German Restaurant,Vietnamese Restaurant
4,Au - Haidhausen,Café,Italian Restaurant,German Restaurant,French Restaurant,Bavarian Restaurant
7,Schwanthalerhöhe,Café,Italian Restaurant,German Restaurant,Asian Restaurant,Ice Cream Shop
17,Untergiesing - Harlaching,Café,German Restaurant,Italian Restaurant,Ice Cream Shop,Bistro


In districts of this cluster families will mostly find `cafe` and `german/bavarian/italian restaurant`.
Notably all the district here are located in the city center, therefore it’s understandable that `cafe` and `restaurant` are the most common places.

#### Cluster 3

In [87]:
show_cluster(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Neuhausen - Nymphenburg,Italian Restaurant,German Restaurant,Café,Bakery,Sushi Restaurant
9,Moosach,Italian Restaurant,German Restaurant,Greek Restaurant,Café,Bakery
12,Bogenhausen,Italian Restaurant,German Restaurant,Café,Restaurant,Trattoria/Osteria
15,Ramersdorf - Perlach,German Restaurant,Italian Restaurant,Café,Greek Restaurant,Restaurant
23,Feldmoching - Hasenbergl,German Restaurant,Greek Restaurant,Café,Trattoria/Osteria,Italian Restaurant


This cluster seems to be similar to `Cluster 1`, with prevailing number of `cafes` vs `bakeries` in `Cluster 1`. Interestingly just like in `Cluster 1` districts of this cluster aren’t located within the same part of the city.

#### Cluster 4

In [89]:
show_cluster(4)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Sendling,Café,Italian Restaurant,Ice Cream Shop,German Restaurant,Greek Restaurant
6,Sendling - Westpark,Café,Italian Restaurant,Greek Restaurant,Ice Cream Shop,Asian Restaurant
16,Obergiesing,Café,German Restaurant,Ice Cream Shop,Italian Restaurant,Restaurant
18,Thalkirchen - Obersendling - Forstenried - Für...,German Restaurant,Italian Restaurant,Café,Greek Restaurant,Ice Cream Shop
19,Hadern,Café,Ice Cream Shop,German Restaurant,Greek Restaurant,Italian Restaurant
24,Laim,Café,Italian Restaurant,German Restaurant,Greek Restaurant,Ice Cream Shop


In districts of this cluster families will mostly find: `cafe`, `italian restaurant`, `german restaurant` and `ice cream shop`.

## Results and Discussion

My analysis shows that districts in Munich have different predominant family-friendly places, and therefore it’s possible to cluster them in various categories.

I learned that most of all, Munich has cafes, Italian restaurants, German restaurants, Greek restaurants, and bakeries.

Cafes are prevailing in the city center, while restaurants are prevailing outside of the city center. Interestingly, only one cluster showed a good amount of ice cream shops (southern region of Munich).

Finally, I’d like to note that the analysis was done on the places returned by Foursquare API with `family-friendly` keyword. Analyzing food places without such constraints may give a different result, but the goal of this project was to focus on family-friendly places.

## Conclusion

This project helps families in Munich to better understand the city districts from the eating places perspective. I grouped similar districts into clusters and showed their difference.
There’s no better cluster, everyone can decide what’s best for them, because each cluster has many food options.
The project can be used as part of a bigger study on Munich districts to find family friendly districts using multiple different parameters, not just eating places.